In [131]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',100)
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split as tts,cross_val_score,ShuffleSplit
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import seaborn as sns

In [132]:
df = pd.read_csv('cust_inv_merged_final.csv')

In [133]:
master_df = df.groupby(['Make','Model','state','City','Order Type']).agg({'Total Amt Wtd Tax.' : 'mean','ServiceHours':'mean'})

In [134]:
master_df.columns = ['Average Revenue','Average Hrs']
master_df.reset_index(inplace=True)
master_df.shape

(36500, 7)

In [135]:
master_df.head()

,Make,Model,state,City,Order Type,Average Revenue,Average Hrs
0,AUDI,A4,Andhra Pradesh,kakinada,Running Repairs,812.353333,10.183333
1,AUDI,A4,Andhra Pradesh,kurnool,Running Repairs,21834.920000,28.300000
2,AUDI,A4,Gujarat,surat,Paid Service,554.630000,2.550000
3,AUDI,A4,Gujarat,vapi,Accidental,2480.490000,2.970000
4,AUDI,A4,Himachal Pradesh,una,Running Repairs,6449.705000,61.122500


##### Consider the year 2016 for calculationg the Life Time vaue of teh customers'

In [136]:
df_2016 = df[df['Year']==2016]

In [137]:
df_2016.shape

(197286, 32)

In [138]:
df_2016.head()

,Unnamed: 0,Invoice No,Job Card No,Customer No.,Cust Type,Partner Type,Order Type,Data Origin,User ID,Plant,Plant Name1,City,state,Zones,Make,Model,Price,Model Category,age,Days,ServiceHours,Service_Time,KMs Reading,Year,Month,Season,Labour Total,Parts Total,OSL Total,Misc Total,Recovrbl Exp,Total Amt Wtd Tax.
295028,295028,7005202789,300412,119371,Retail,1,SMC Value Package,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,ZEN ESTILO,340000,Budget Friendly,>5,0,0.58,0 days 00:35:03.000000000,35654,2016,1,Winter,1125.76,2082.07,0.0,291.45,0.0,3499.28
295029,295029,7005202790,300447,128929,Retail,1,Running Repairs,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,SWIFT,519000,Budget Friendly,>5,0,1.20,0 days 01:12:18.000000000,70730,2016,1,Winter,233.85,195.44,0.0,0.00,0.0,429.29
295030,295030,7005202791,300443,119371,Retail,1,SMC Redemption,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,ZEN ESTILO,340000,Budget Friendly,>5,0,3.13,0 days 03:08:31.000000000,163215,2016,1,Winter,0.00,0.00,0.0,0.00,0.0,0.00
295031,295031,7005202792,300505,E10724,Retail,1,Running Repairs,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,FIAT,PALIO,358000,Budget Friendly,>5,0,3.60,0 days 03:36:50.000000000,50220,2016,1,Winter,69.97,297.65,0.0,0.00,0.0,367.62
295032,295032,7005202793,300561,92107,Retail,1,Paid Service,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,SWIFT,519000,Budget Friendly,>5,0,3.28,0 days 03:17:21.000000000,79061,2016,1,Winter,698.64,44.44,0.0,0.00,0.0,743.08


In [139]:
merged_df = pd.merge(df_2016,master_df,on=['Make','Model','state','City','Order Type'],how='left')

In [140]:
merged_df.shape

(197286, 34)

In [141]:
merged_df.head()

,Unnamed: 0,Invoice No,Job Card No,Customer No.,Cust Type,Partner Type,Order Type,Data Origin,User ID,Plant,Plant Name1,City,state,Zones,Make,Model,Price,Model Category,age,Days,ServiceHours,Service_Time,KMs Reading,Year,Month,Season,Labour Total,Parts Total,OSL Total,Misc Total,Recovrbl Exp,Total Amt Wtd Tax.,Average Revenue,Average Hrs
0,295028,7005202789,300412,119371,Retail,1,SMC Value Package,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,ZEN ESTILO,340000,Budget Friendly,>5,0,0.58,0 days 00:35:03.000000000,35654,2016,1,Winter,1125.76,2082.07,0.0,291.45,0.0,3499.28,3166.237333,23.585333
1,295029,7005202790,300447,128929,Retail,1,Running Repairs,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,SWIFT,519000,Budget Friendly,>5,0,1.20,0 days 01:12:18.000000000,70730,2016,1,Winter,233.85,195.44,0.0,0.00,0.0,429.29,2485.665090,45.172803
2,295030,7005202791,300443,119371,Retail,1,SMC Redemption,Z005,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,ZEN ESTILO,340000,Budget Friendly,>5,0,3.13,0 days 03:08:31.000000000,163215,2016,1,Winter,0.00,0.00,0.0,0.00,0.0,0.00,60.363636,150.801818
3,295031,7005202792,300505,E10724,Retail,1,Running Repairs,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,FIAT,PALIO,358000,Budget Friendly,>5,0,3.60,0 days 03:36:50.000000000,50220,2016,1,Winter,69.97,297.65,0.0,0.00,0.0,367.62,2708.158077,112.535769
4,295032,7005202793,300561,92107,Retail,1,Paid Service,Z001,BC01SA2,BC01,THANE,thane,Maharashtra,West,MARUTI SUZUKI,SWIFT,519000,Budget Friendly,>5,0,3.28,0 days 03:17:21.000000000,79061,2016,1,Winter,698.64,44.44,0.0,0.00,0.0,743.08,4085.619921,63.472823


In [142]:
customer_df = merged_df.groupby(['Make','state','Order Type','Customer No.']).agg(
    {"Invoice No":'nunique','Average Revenue':'mean','Average Hrs':'mean'})

In [143]:
customer_df.columns = ['Frequency','Average Revenue','Average hrs']

In [144]:
customer_df.reset_index(inplace=True)

In [145]:
customer_df.shape

(160391, 7)

In [146]:
customer_df['Customer No.'].nunique()

131243

In [147]:
customer_df.head()

,Make,state,Order Type,Customer No.,Frequency,Average Revenue,Average hrs
0,AUDI,Andhra Pradesh,Accidental,215835,1,123721.580000,20.180000
1,AUDI,Andhra Pradesh,Running Repairs,182988,2,812.353333,10.183333
2,AUDI,Andhra Pradesh,Running Repairs,198994,3,434.740000,1.556667
3,AUDI,Andhra Pradesh,Running Repairs,204866,1,434.740000,2.450000
4,AUDI,Andhra Pradesh,Running Repairs,211383,1,1619.140000,293.570000


In [148]:
customer_df['LTV'] = round((customer_df['Frequency'] * customer_df['Average Revenue']),2)

In [149]:
customer_df =  customer_df[customer_df['LTV']>0]

In [150]:
encode=pd.get_dummies(data=customer_df, columns=['Make','state','Order Type'])

In [151]:
final_df=pd.DataFrame(encode)

In [152]:
final_df.shape

(158548, 62)

In [153]:
final_df.head()

,Customer No.,Frequency,Average Revenue,Average hrs,LTV,Make_AUDI,Make_BMW,Make_DAEWOO,Make_FIAT,Make_FORCE,Make_FORD,Make_GENERAL MOTORS,Make_HINDUSTAN MOTORS,Make_HONDA,Make_HYUNDAI,Make_JEEP,Make_LAND ROVER,Make_MAHINDRA & MAHINDRA,Make_MARUTI SUZUKI,Make_MERCEDES BENZ,Make_MITSUBISHI MOTORS,Make_MORRIS,Make_NISSAN,Make_PORCHE,Make_PREMIER AUTOMOBILES,Make_RENAULT,Make_SAN MOTORS,Make_SKODA,Make_SONALIKA,Make_TATA MOTORS,Make_TOYOTA,Make_VOLKSWAGEN,Make_VOLVO,state_Andhra Pradesh,state_Assam,state_Bihar,state_Chandigarh,state_Chhattisgarh,state_Gujarat,state_Haryana,state_Himachal Pradesh,state_Jharkhand,state_Karnataka,state_Kerala,state_Madhya Pradesh,state_Maharashtra,state_Odisha,state_Puducherry,state_Punjab,state_Rajasthan,state_Tamil Nadu,state_Telangana,state_Uttar Pradesh,state_Uttarakhand,state_West Bengal,Order Type_Accidental,Order Type_Mechanical,Order Type_Paid Service,Order Type_Running Repairs,Order Type_SMC Redemption,Order Type_SMC Value Package,Order Type_WBW Order
0,215835,1,123721.580000,20.180000,123721.58,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,182988,2,812.353333,10.183333,1624.71,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,198994,3,434.740000,1.556667,1304.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,204866,1,434.740000,2.450000,434.74,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,211383,1,1619.140000,293.570000,1619.14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [154]:
model_df =  final_df.copy()

In [123]:
# le=LabelEncoder()
# model_df["Make"]=le.fit_transform(model_df["Make"])
# model_df["Model"]=le.fit_transform(model_df["Model"])
# #model_df["state"]=le.fit_transform(model_df["state"])
# model_df["City"]=le.fit_transform(model_df["City"])
# model_df["Order Type"]=le.fit_transform(model_df["Order Type"])

In [155]:
model_df.head()

,Customer No.,Frequency,Average Revenue,Average hrs,LTV,Make_AUDI,Make_BMW,Make_DAEWOO,Make_FIAT,Make_FORCE,Make_FORD,Make_GENERAL MOTORS,Make_HINDUSTAN MOTORS,Make_HONDA,Make_HYUNDAI,Make_JEEP,Make_LAND ROVER,Make_MAHINDRA & MAHINDRA,Make_MARUTI SUZUKI,Make_MERCEDES BENZ,Make_MITSUBISHI MOTORS,Make_MORRIS,Make_NISSAN,Make_PORCHE,Make_PREMIER AUTOMOBILES,Make_RENAULT,Make_SAN MOTORS,Make_SKODA,Make_SONALIKA,Make_TATA MOTORS,Make_TOYOTA,Make_VOLKSWAGEN,Make_VOLVO,state_Andhra Pradesh,state_Assam,state_Bihar,state_Chandigarh,state_Chhattisgarh,state_Gujarat,state_Haryana,state_Himachal Pradesh,state_Jharkhand,state_Karnataka,state_Kerala,state_Madhya Pradesh,state_Maharashtra,state_Odisha,state_Puducherry,state_Punjab,state_Rajasthan,state_Tamil Nadu,state_Telangana,state_Uttar Pradesh,state_Uttarakhand,state_West Bengal,Order Type_Accidental,Order Type_Mechanical,Order Type_Paid Service,Order Type_Running Repairs,Order Type_SMC Redemption,Order Type_SMC Value Package,Order Type_WBW Order
0,215835,1,123721.580000,20.180000,123721.58,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,182988,2,812.353333,10.183333,1624.71,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,198994,3,434.740000,1.556667,1304.22,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,204866,1,434.740000,2.450000,434.74,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,211383,1,1619.140000,293.570000,1619.14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [156]:
X = model_df.drop(['Customer No.','LTV'],1)
y = model_df['LTV']

In [157]:
X.columns

Index(['Frequency', 'Average Revenue', 'Average hrs', 'Make_AUDI', 'Make_BMW',
       'Make_DAEWOO', 'Make_FIAT', 'Make_FORCE', 'Make_FORD',
       'Make_GENERAL MOTORS', 'Make_HINDUSTAN MOTORS', 'Make_HONDA',
       'Make_HYUNDAI', 'Make_JEEP', 'Make_LAND ROVER',
       'Make_MAHINDRA &  MAHINDRA', 'Make_MARUTI SUZUKI', 'Make_MERCEDES BENZ',
       'Make_MITSUBISHI MOTORS', 'Make_MORRIS', 'Make_NISSAN', 'Make_PORCHE',
       'Make_PREMIER AUTOMOBILES', 'Make_RENAULT', 'Make_SAN MOTORS',
       'Make_SKODA', 'Make_SONALIKA', 'Make_TATA MOTORS', 'Make_TOYOTA',
       'Make_VOLKSWAGEN', 'Make_VOLVO', 'state_Andhra Pradesh', 'state_Assam',
       'state_Bihar', 'state_Chandigarh', 'state_Chhattisgarh',
       'state_Gujarat', 'state_Haryana', 'state_Himachal Pradesh',
       'state_Jharkhand', 'state_Karnataka', 'state_Kerala',
       'state_Madhya Pradesh', 'state_Maharashtra', 'state_Odisha',
       'state_Puducherry', 'state_Punjab', 'state_Rajasthan',
       'state_Tamil Nadu', 'state

In [158]:
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.3,random_state=17)

#### Linear Regression Base Line Model

In [159]:
lr = LinearRegression()

lr.fit(X_train,y_train)

y_pred_lr = lr.predict(X_test)

mse =mean_squared_error(y_test,y_pred_lr)
print('mse score:',mse)
print('=='*100)
rmse=mean_squared_error(y_test,y_pred_lr)
rmse=np.sqrt(rmse)
print('Rmse score:',rmse)

#accuracy score on train data
train_score=lr.score(X_train,y_train)
print('train score:',train_score)
print('=='*100)

#accuracy score on test data
test_score=lr.score(X_test,y_test)
print('test score:',test_score)
print('=='*100)

R2_Score=r2_score(y_test,y_pred_lr)
print('R2 score:',R2_Score)
print('=='*100)

mse score: 51579074.37634263
Rmse score: 7181.857306877006
train score: 0.7061036000951015
test score: 0.6206153911688566
R2 score: 0.6206153911688566


#### Decision Tree Regressor Base Line Model

In [160]:
dt = DecisionTreeRegressor()

dt.fit(X_train,y_train)

y_pred_dt = dt.predict(X_test)

mse =mean_squared_error(y_test,y_pred_dt)
print('mse score:',mse)
print('=='*100)
rmse=mean_squared_error(y_test,y_pred_dt)
rmse=np.sqrt(rmse)
print('Rmse score:',rmse)

#accuracy score on train data
train_score=dt.score(X_train,y_train)
print('train score:',train_score)
print('=='*100)

#accuracy score on test data
test_score=dt.score(X_test,y_test)
print('test score:',test_score)
print('=='*100)

R2_Score=r2_score(y_test,y_pred_dt)
print('R2 score:',R2_Score)
print('=='*100)

mse score: 3977462.031894661
Rmse score: 1994.3575486593825
train score: 1.0
test score: 0.9707441846260972
R2 score: 0.9707441846260972


In [161]:
dt1 = DecisionTreeRegressor(random_state=2)
param_grid = { 
'max_depth' : range(2,11),
'min_samples_split': [2,4,6]
}
rs_model = RandomizedSearchCV(dt1, param_distributions=param_grid)
rs_model.fit(X_train, y_train)

rs_model.best_params_

{'min_samples_split': 4, 'max_depth': 9}

In [162]:
dt_best = DecisionTreeRegressor(max_depth=9,min_samples_split=4,random_state=18)

dt_best.fit(X_train,y_train)

y_pred_dt = dt_best.predict(X_test)

mse =mean_squared_error(y_test,y_pred_dt)
print('mse score:',mse)
print('=='*100)
rmse=mean_squared_error(y_test,y_pred_dt)
rmse=np.sqrt(rmse)
print('Rmse score:',rmse)

#accuracy score on train data
train_score=dt_best.score(X_train,y_train)
print('train score:',train_score)
print('=='*100)

#accuracy score on test data
test_score=dt_best.score(X_test,y_test)
print('test score:',test_score)
print('=='*100)

R2_Score=r2_score(y_test,y_pred_dt)
print('R2 score:',R2_Score)
print('=='*100)

mse score: 4271773.573786295
Rmse score: 2066.826933680296
train score: 0.9979998285943119
test score: 0.968579406166128
R2 score: 0.9685794061661281


In [163]:
scores = cross_val_score(dt_best, X, y, cv=5, scoring='r2')
print('scores',scores)
print('=='*100)

print('Mean_score:',scores.mean())
print('=='*100)

print('std_score:',scores.std())

scores [0.97154115 0.97882163 0.96111796 0.97729185 0.9930962 ]
Mean_score: 0.9763737585837795
std_score: 0.01041699186677378


#### Random Forest Regressor Base Line Model

In [164]:
rf = RandomForestRegressor()

rf.fit(X_train,y_train)

y_pred_rf = rf.predict(X_test)

mse =mean_squared_error(y_test,y_pred_rf)
print('mse score:',mse)
print('=='*100)
rmse=mean_squared_error(y_test,y_pred_rf)
rmse=np.sqrt(rmse)
print('Rmse score:',rmse)

#accuracy score on train data
train_score=rf.score(X_train,y_train)
print('train score:',train_score)
print('=='*100)

#accuracy score on test data
test_score=rf.score(X_test,y_test)
print('test score:',test_score)
print('=='*100)

R2_Score=r2_score(y_test,y_pred_rf)
print('R2 score:',R2_Score)
print('=='*100)

mse score: 22921115.098877985
Rmse score: 4787.600139827677
train score: 0.9782507220803708
test score: 0.8314060810337083
R2 score: 0.8314060810337083


In [165]:
rfc = RandomForestRegressor(random_state=2)
param_grid = { 
'max_depth' : range(2,8),
'min_samples_split': [2,4,6]
}
grid_search_model = GridSearchCV(rfc, param_grid=param_grid)
grid_search_model.fit(X_train, y_train)

grid_search_model.best_params_

{'max_depth': 7, 'min_samples_split': 2}

In [166]:
rf_best = RandomForestRegressor(max_depth=7,min_samples_split=2,random_state=17)

rf_best.fit(X_train,y_train)

y_pred_rf = rf_best.predict(X_test)

mse =mean_squared_error(y_test,y_pred_rf)
print('mse score:',mse)
print('=='*100)
rmse=mean_squared_error(y_test,y_pred_rf)
rmse=np.sqrt(rmse)
print('Rmse score:',rmse)

#accuracy score on train data
train_score=rf_best.score(X_train,y_train)
print('train score:',train_score)
print('=='*100)

#accuracy score on test data
test_score=rf_best.score(X_test,y_test)
print('test score:',test_score)
print('=='*100)

R2_Score=r2_score(y_test,y_pred_rf)
print('R2 score:',R2_Score)
print('=='*100)

mse score: 20876411.778296046
Rmse score: 4569.071216154991
train score: 0.970652769103473
test score: 0.8464456872855514
R2 score: 0.8464456872855514


In [167]:
scores = cross_val_score(rf_best, X, y, cv=5, scoring='r2')
print('scores',scores)
print('=='*100)

print('Mean_score:',scores.mean())
print('=='*100)

print('std_score:',scores.std())

scores [0.97098266 0.86289084 0.9405025  0.99381511 0.87660406]
Mean_score: 0.9289590336773175
std_score: 0.05140349556907242
